In [ ]:
# load libraries
import numpy as np
from matplotlib import pylab as plt
import pandas as pd
%matplotlib inline
import glob

import os

## Functions for generating perfromance stats

In [ ]:
def getstats(s1,s2):
    """ find probability that the random item in s1 is smaller/larger than the one in s2 """
    cntless=0
    cntmore=0
    
    for a in s1:
        for b in s2:
            if a<b:
                cntless = cntless+1
            elif a>b:
                cntmore = cntmore+1                             
    return cntless/(s1.shape[0]*s2.shape[0]), cntmore/(s1.shape[0]*s2.shape[0])

def bootstrap(A,B):
    """ generate a number of bootstrap samples of statistic generated by getstats()"""
    bootstraps = 9999
    statless=[]
    statmore=[]
    for i in range(bootstraps):
        sample1 = np.random.choice(A,size=A.shape[0],replace=True)
        sample2 = np.random.choice(B,size=B.shape[0],replace=True)
        cntless, cntmore = getstats(sample1,sample2)
        statless.append(cntless)
        statmore.append(cntmore)
    return statless,statmore

## Select problems for comparison

In [ ]:
# generate problem names for comparison
problems = []
for i in range(41,51):
    problems.append("ta{}".format(i))
    
 
# problems dmu41:dmu50
problems = ["cscmax_50_15_10",  "cscmax_50_15_4",  "cscmax_50_15_8",  "cscmax_50_20_3",  "cscmax_50_20_7","cscmax_50_15_3",
           "cscmax_50_15_6",  "cscmax_50_20_1",  "cscmax_50_20_4",  "cscmax_50_20_9"]

# problems dmu51:dmu60
#problems=["cscmax_40_15_3", "cscmax_40_15_6", "cscmax_40_15_8", "cscmax_40_15_4",
#          "cscmax_40_15_7", "cscmax_40_20_10", "cscmax_40_20_6", "cscmax_40_20_8",
#          "cscmax_40_20_5", "cscmax_40_20_9"]

#problems dmu61:dmu71
#problems =["cscmax_30_15_2","cscmax_30_15_9", "cscmax_30_15_10", "cscmax_30_15_5",
#             "cscmax_30_15_6", "cscmax_30_20_9" ,"cscmax_30_20_7", "cscmax_30_20_3",
#             "cscmax_30_20_6", "cscmax_30_20_4"]

# problems dmu71:dmu80
#problems = ["cscmax_20_15_10", "cscmax_20_15_5", "cscmax_20_15_8", "cscmax_20_15_7", "cscmax_20_15_1",
#            "cscmax_20_20_6", "cscmax_20_20_4", "cscmax_20_20_3", "cscmax_20_20_2",  "cscmax_20_20_9"]
            

print("will generate probability plots for \n",problems)

## Select algorithm/expid for comparison

In [ ]:

# algoname is the name of the algorithm (e.g., gta)
# expit is the name of the experiment (e.g. =gtacurve= is the gta algorithm with theta loguniformly distributed)

datapath = 'data/performance' # root folder for the performance data
# the folder structure is data/performance/algoname/experimentid

colnames=['algo','expid','problem','runid','mpirank','seed','obj','time','epoch']

# DATA ON ALGO A
datasetX = pd.DataFrame([],columns=colnames)
algo1name = 'gta'
exp1name = 'gtacurve'
for problem in problems:
    directory = "{}/{}/{}/{}".format(datapath,algo1name,exp1name,problem)
    lst = glob.glob('{}/*'.format(directory)) #
    for file in lst:
        data = pd.read_csv("{}".format(file),header=None, 
                   names=colnames)
        datasetX = pd.concat( [datasetX,data] )
        
# DATA ON ALGO B   
algo2name = 'stabu'
exp2name = 'tabu0'
datasetY = pd.DataFrame([],columns=colnames)
for problem in problems:
    directory = "{}/{}/{}/{}".format(datapath,algo2name,exp2name,problem)
    lst = glob.glob('{}/*'.format(directory)) #    
    for file in lst:
        data = pd.read_csv("{}".format(file),header=None, 
                   names=colnames)
        datasetY = pd.concat( [datasetY,data] )
        
print("the sizes of two datasets are:", datasetX.shape, datasetY.shape)

# Probability dominanace plots with respect to the number of iterations.

In [ ]:
xl, yl, ll, ul, xg, yg, lg, ug = [], [], [], [], [], [] , [], []

bestobjX = 99999999999999
bestobjY = 99999999999999

Lprobless={}
Uprobless={}
Mprobless={}

Lprobgrt={}
Uprobgrt={}
Mprobgrt={}

bestobjX={}
bestobjY={}

epochstart = 0
epochend = 201
step = 1

for epoch in range(epochstart,epochend,step):       
    statless= []
    statgrt = []
    pless = 0
    pgrt = 0
    for problem in problems: 
        # get the best objective found up to epoch and group by random seed 
        objX = np.array(datasetX[ (datasetX.problem=="{}.txt".format(problem)) & 
                                 (datasetX.epoch<=epoch)].groupby( ['seed'])['obj'].min())
        objY = np.array(datasetY[ (datasetY.problem=="{}.txt".format(problem)) & 
                                 (datasetY.epoch<=epoch)].groupby( ['seed'])['obj'].min())
        
        pl,pg = getstats(objX,objY) # find probability less and greater
        sless,sgrt = bootstrap(objX,objY) # bootstrap to get error estimates
        statless.append(sless)
        statgrt.append(sgrt)
        pless = pless + pl
        pgrt = pgrt + pg
    # find P< and 95% CI
    npstatless = np.array(statless)
    data = npstatless.mean(axis=0) # average across all problems    
    lower = np.percentile(data, 2.5)
    upper = np.percentile(data, 97.5)  
    Lprobless[epoch]=lower
    Uprobless[epoch]=upper
    Mprobless[epoch]=1.0*pless/len(problems)    
    
    # finf P> and 95% CI
    npstatgrt = np.array(statgrt)
    data = npstatgrt.mean(axis=0)    
    lower = np.percentile(data, 2.5)
    upper = np.percentile(data, 97.5)  
    Lprobgrt[epoch]=lower
    Uprobgrt[epoch]=upper
    Mprobgrt[epoch]=1.0*pgrt/len(problems)
    print("at epoch {} P(f{}<f{})={:.2f} and P(f{}>f{})={:.2f}".format(epoch, algo1name,algo2name,Mprobless[epoch],
                                                           algo1name,algo2name, Mprobgrt[epoch]))
    

In [ ]:
# consolidate data
#multiplyer = 1.0/len(problems)
xl, yl, ll, ul, xg, yg, lg, ug = [], [], [], [], [], [] , [], []    
for epoch in range(0,201,1):    
    lless=Lprobless[epoch] 
    uless=Uprobless[epoch] 
    mless=Mprobless[epoch] 
    lgreat=Lprobgrt[epoch] 
    ugreat=Uprobgrt[epoch] 
    mgreat=Mprobgrt[epoch] 
        
    xl.append(epoch)
    yl.append(mless)
    ll.append(lless)
    ul.append(uless)
    
    xg.append(epoch)
    lg.append(lgreat)
    ug.append(ugreat)
    yg.append(mgreat)

    
fig, axs = plt.subplots(1, 1)    
axs.set_yticks(np.arange(0, 1.05, 0.1))
plt.plot(xl,yl,ls="dashed",label=r'Probability that $gta$ finds better sol: $P[f_{gta}< f_{tabu)}]$')
plt.plot(xg,yg,label=r'Probability that $tabu$ finds better sol: $P[f_{tabu}<f_{gta}]$ ')

axs.text(0.1,0.8, "Demirkol's Instances dmu71-dmu80")
                                                                                   
plt.ylim(0,1.2)
#plt.fill_between(x, l, u, facecolor='#F0F8FF', alpha=1.0, edgecolor='none')
plt.fill_between(xl, ll, ul, facecolor='#97c6ef', alpha=0.6, edgecolor='none')
plt.fill_between(xg, lg, ug, facecolor='#e8d688', alpha=0.6, edgecolor='none')
plt.xlabel("epoch")
plt.ylabel("Probability")
#plt.axhline(y=0.5,linestyle='--',linewidth=0.5, color='r')
plt.grid()
plt.legend(loc='upper center')#, bbox_to_anchor=(0.42, 1.25))
plt.savefig("figs/dmu71-dmu80-{}-vs-{}.png".format(exp1name,exp2name))#,dpi=1200)
#plt.close()

# Probability dominanace plots with respect to the computational time.

In [ ]:
xl, yl, ll, ul, xg, yg, lg, ug = [], [], [], [], [], [] , [], []

Lprobless={}
Uprobless={}
Mprobless={}

Lprobgrt={}
Uprobgrt={}
Mprobgrt={}

maxtime = 1800
step = 60
minute = 0
for time in range(60,maxtime,step):           
    minute = minute + 1
    statless= []
    statgrt = []
    pless = 0
    pgrt = 0
    for problem in problems:            
        objX = np.array(datasetX[ (datasetX.problem=="{}.txt".format(problem)) & (datasetX.time<=time)].groupby( ['seed'])['obj'].min())
        objY = np.array(datasetY[ (datasetY.problem=="{}.txt".format(problem)) & (datasetY.time<=time)].groupby( ['seed'])['obj'].min())
        pl,pg = getstats(objX,objY) # find probability less and greater
        sless,sgrt = bootstrap(objX,objY) # bootstrap to get error estimates
        statless.append(sless)
        statgrt.append(sgrt)
        pless = pless + pl
        pgrt = pgrt + pg
    # find P< and 95% CI
    npstatless = np.array(statless)
    data = npstatless.mean(axis=0) # average across all problems    
    lower = np.percentile(data, 2.5)
    upper = np.percentile(data, 97.5)  
    Lprobless[minute]=lower
    Uprobless[minute]=upper
    Mprobless[minute]=1.0*pless/len(problems)    
    
    # finf P> and 95% CI
    npstatgrt = np.array(statgrt)
    data = npstatgrt.mean(axis=0)    
    lower = np.percentile(data, 2.5)
    upper = np.percentile(data, 97.5)  
    Lprobgrt[minute]=lower
    Uprobgrt[minute]=upper
    Mprobgrt[minute]=1.0*pgrt/len(problems)
    print("at minute {} P(f{}<f{})={:.2f} and P(f{}>f{})={:.2f}".format(minute, algo1name,algo2name,Mprobless[minute],
                                                           algo1name,algo2name, Mprobgrt[minute]))      

In [ ]:
xl, yl, ll, ul, xg, yg, lg, ug = [], [], [], [], [], [] , [], []    
minute = 0
for time in range(60,maxtime,step):           
    minute = minute + 1    
    lless=Lprobless[minute] 
    uless=Uprobless[minute] 
    mless=Mprobless[minute] 
    lgreat=Lprobgrt[minute] 
    ugreat=Uprobgrt[minute] 
    mgreat=Mprobgrt[minute] 
    
    xl.append(minute)
    yl.append(mless)
    ll.append(lless)
    ul.append(uless)
    
    xg.append(minute)
    lg.append(lgreat)
    ug.append(ugreat)
    yg.append(mgreat)

    
fig, axs = plt.subplots(1, 1)    
axs.set_yticks(np.arange(0, 1.05, 0.1))
plt.plot(xl,yl,ls="dashed",label=r'Probability that $gta$ finds better sol: $P[f_{gta}< f_{tabu)}]$')
plt.plot(xg,yg,label=r'Probability that $tabu$ finds better sol: $P[f_{tabu}<f_{gta}]$ ')

axs.text(0.1,0.8, "Taillard's Instances ta21-ta31")
                                                                                   
plt.ylim(0,1.2)
#plt.fill_between(x, l, u, facecolor='#F0F8FF', alpha=1.0, edgecolor='none')
plt.fill_between(xl, ll, ul, facecolor='#97c6ef', alpha=0.6, edgecolor='none')
plt.fill_between(xg, lg, ug, facecolor='#e8d688', alpha=0.6, edgecolor='none')
plt.xlabel("time, minutes")
plt.ylabel("Probability")
#plt.axhline(y=0.5,linestyle='--',linewidth=0.5, color='r')
plt.grid()
plt.legend(loc='upper center')#, bbox_to_anchor=(0.42, 1.25))
plt.savefig("figs/ta21-ta30-{}-vs-{}-TIME.png".format(exp1name,exp2name))#,dpi=1200)
#plt.close()